In [1]:
import pandas as pd
import matplotlib as mp
import re
import plotly.plotly as py
import plotly.graph_objs as go
import sklearn as sk


In [2]:
df_train = pd.read_csv('Data/train.csv', encoding='ISO-8859-1')
df_attr = pd.read_csv('Data/attributes.csv',  encoding='ISO-8859-1')
df_prod_desc = pd.read_csv('Data/attributes.csv', encoding='ISO-8859-1')

In [3]:
df_attr[df_attr.name.str.contains('.Color.') == True].groupby('value').count()

,product_uid,name
value,,
.Combination Pack,2,2
0,99,99
000,1,1
1,5,5
1800,1,1
2000,2,2
2200,11,11
22000,1,1
2300,1,1


In [4]:
def getPercStr1inStr2(str1, str2):
    # tokenized_str1 = nltk.tokenize.word_tokenize(str(str1))
    str2 = re.sub(r'[^\x00-\x7F]+',' ', str2)
    tokenized_str1 = str1.split()
    return sum([int(word in str2) for word in tokenized_str1])/float(len(tokenized_str1))

In [5]:
getPercStr1inStr2(df_train['search_term'][0], df_train['product_title'][0])

0.0

In [6]:
df_train['search_in_title'] = list(map(lambda x: getPercStr1inStr2(df_train['search_term'][x], df_train['product_title'][x]), range(df_train.shape[0])))

In [9]:
df_trainedModel = pd.read_table('Data/train_with_features.csv', sep="|", quotechar=" ", encoding='ISO-8859-1')


In [12]:
df_trainedModel.columns

Index(['product_uid', 'id', 'product_title', 'search_term', 'relevance',
       'product_description', 'perc_term_in_title', 'perc_term_in_descr',
       'whole_term_in_title', 'whole_term_in_descr', 'search_term_length'],
      dtype='object')

In [11]:
#Renaming to remove the quotes from the beginning of the col names
df_trainedModel.rename(columns=lambda x: x[:-1], inplace=True)
df_trainedModel.rename(columns=lambda x: x[1:], inplace=True)

In [13]:
df_trainedModel_data = df_trainedModel[['perc_term_in_title','perc_term_in_descr','whole_term_in_title','whole_term_in_descr','search_term_length']]

In [14]:
import math
df_trainedModel_target = df_trainedModel['relevance'].map(lambda x: math.floor(x))

In [16]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
new_model = gnb.fit(df_trainedModel_data, df_trainedModel_target)

In [17]:
y_pred= new_model.predict(df_trainedModel_data)

In [18]:
print("Number of mislabeled points out of a total %d points : %d" % (df_trainedModel_data.shape[0],(df_trainedModel_target != y_pred).sum()))

Number of mislabeled points out of a total 74067 points : 32285


In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df_trainedModel_target, y_pred)

array([[ 2201,  9067,   641],
       [ 3731, 35898,  3404],
       [ 1161, 14281,  3683]])

In [36]:
import numpy as np
from sklearn import cross_validation
train_data, test_data, train_target, test_target = cross_validation.train_test_split \
                    (df_trainedModel_data, df_trainedModel_target, \
                    test_size = 0.3, random_state = 0)

In [37]:
train_data.shape, train_target.shape

((51846, 5), (51846,))

In [39]:
test_data.shape, test_target.shape

((22221, 5), (22221,))

In [40]:
newM = gnb.fit(train_data, train_target)
newM.score(test_data, test_target)

0.56689617928986091

In [45]:
scores = cross_validation.cross_val_score(gnb, df_trainedModel_data, df_trainedModel_target, cv=10)
scores

array([ 0.5537257 ,  0.54846112,  0.5449514 ,  0.54961523,  0.5436749 ,
        0.54847421,  0.54590872,  0.62975965,  0.61139617,  0.56394328])

In [46]:
print("Gaussian Naive Bayes Model using 10 fold CV ")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Gaussian Naive Bayes Model using 5 fold CV 
Accuracy: 0.56 (+/- 0.06)


In [49]:
from sklearn import metrics
predicted = cross_validation.cross_val_predict(gnb, df_trainedModel_data, df_trainedModel_target, cv=10)
metrics.accuracy_score(df_trainedModel_target, predicted)

0.56398936098397401

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

clf = RandomForestClassifier(n_estimators=100)
clf.fit(train_data, train_target)
clf_probs = clf.predict_proba(test_data)
score = log_loss(test_target, clf_probs)

In [60]:
clf.score(test_data, test_target)

0.59502272624994379

In [62]:
regModel_data = df_trainedModel[['perc_term_in_title','perc_term_in_descr','whole_term_in_title','whole_term_in_descr','search_term_length']]
regModel_target = df_trainedModel['relevance']
rtrain_data, rtest_data, rtrain_target, rtest_target = cross_validation.train_test_split \
                    (regModel_data, regModel_target, \
                    test_size = 0.3, random_state = 0)

In [63]:
rtrain_data.shape, rtrain_target.shape

((51846, 5), (51846,))

In [79]:
from sklearn.ensemble import RandomForestRegressor
clf2 = RandomForestRegressor(n_estimators=100)
clf2.fit(rtrain_data, rtrain_target)
#clf2.score(rtest_data, rtest_target)
predicted = cross_validation.cross_val_predict(clf2, rtrain_data, rtrain_target, cv=10)

In [80]:
predicted

array([ 2.76254589,  2.27770788,  2.55455742, ...,  2.65615442,
        2.30753567,  2.21159133])

In [81]:
metrics.mean_squared_error(rtrain_target, predicted)

0.25629944663868492

In [82]:
predicted = cross_validation.cross_val_predict(clf2, rtest_data, rtest_target, cv=10)

In [83]:
metrics.mean_squared_error(rtest_target, predicted)

0.25717455082128488

In [84]:
rtrain_data

,perc_term_in_title,perc_term_in_descr,whole_term_in_title,whole_term_in_descr,search_term_length
"""34506""",1.000000,1.000000,True,False,17
"""66441""",0.333333,0.333333,False,False,21
"""17902""",1.000000,1.000000,True,True,7
"""46210""",0.166667,0.333333,False,False,37
"""71029""",0.666667,0.666667,False,False,23
"""45040""",1.000000,1.000000,False,False,14
"""10772""",0.500000,0.750000,False,False,21
"""25447""",1.000000,1.000000,False,False,12
"""13295""",0.000000,0.000000,False,False,6
"""791""",1.000000,1.000000,True,True,7


In [86]:
df_attr.head()

,product_uid,name,value
0,100001,Bullet01,Versatile connector for various 90Â° connectio...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001,Bullet03,Help ensure joints are consistently straight a...
3,100001,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001,Bullet05,Made from 12-Gauge steel
